#CPU - Prostokąty

In [ ]:
%%cu
#include<stdio.h>
#include<time.h>

float dokladnosc(int N) //# zmienna określająca dokładność całkowania
{
float sum = 0; 
for(int i = 0; i < N; ++i)
{
float x = (float) i / (float) N;
float fx = (x*x+x+2);
sum += fx;
}
sum *= 1.0f / (float)N;
return sum;
}
int main(){
    FILE *fp = fopen("prostokatcpu1000000.txt", "w");
    
    clock_t start, stop;
    float calka ; 
    start = clock();
    calka =  dokladnosc(1000000);
    stop = clock();
    printf("wartość całki: %f\n", calka);
    printf("czas wykonywania: %lf s", (double)(stop - start) / CLOCKS_PER_SEC);
    fprintf(fp, "%f\n", (double)(stop - start) / CLOCKS_PER_SEC);
    fclose(fp);
    return 0 ;  
}

# GPU - Prostokąty

In [ ]:
%%cu
#include <cuda_runtime.h>
#include<stdio.h>
__global__ void integratorKernel(float *a, int N)
//#est definicją funkcji kernela CUDA, który jest uruchamiany na urządzeniu GPU. Funkcja ta przyjmuje dwa argumenty: wskaźnik a na tablicę typu float i wartość N oznaczającą liczbę elementów w tej tablicy.
{
int idx = blockIdx.x * blockDim.x + threadIdx.x;
 //#jest wyrażeniem, które określa indeks wątku (thread index) dla każdego wątku. blockIdx.x i threadIdx.x to zmienne CUDA, które są automatycznie ustawiane przez CUDA i określają położenie wątku w bloku i w wierszu w bloku, odpowiednio.
float x = (float)idx /(float)N;
 //# jest wyrażeniem, które oblicza wartość x jako współczynnik proporcjonalny do idx i N. Wartość ta jest używana do obliczenia wartości a[idx].
if(idx < N)
 //# jest instrukcją warunkową, która oblicza wartość a[idx] jako (x*x+x+2) tylko wtedy, gdy idx jest mniejsze od N. To zapewnia, że wątek nie próbuje odwołać się do nieistniejącego elementu tablicy.
a[idx] = (x*x+x+2);
 
}
float licz(int N)
 // # jest definicją funkcji licz, która jest główną funkcją programu. Funkcja ta oblicza wartość całki z funkcji zdefinicji
{
size_t size = N * sizeof(float);
float* a_h = (float *)malloc(size);
 //#  tworzymy wskaźnik a_h i alokujemy pamięć dla N elementów typu float.
float* a_d ; cudaMalloc((void **) &a_d, size);
 // #tworzymy wskaźnik a_d i alokujemy pamięć na karcie graficznej (GPU) dla N elementów typu float.
int block_size = 256;
 //#  ustawiamy rozmiar bloku wątków na 256.
int n_blocks = N/block_size + (N % block_size == 0 ? 0:1);
 //# obliczamy ilość bloków wątków, które będą potrzebne do przetworzenia całego zakresu N.
integratorKernel <<< n_blocks, block_size >>> (a_d, N);
 // #uruchamiamy funkcję integratorKernel na GPU z n_blocks blokami wątków, każdy z rozmiarem block_size. Funkcja ta wpisuje wartości do pamięci GPU.
cudaMemcpy(a_h, a_d, sizeof(float)*N, cudaMemcpyDeviceToHost);
 // #kopiujemy wartości z pamięci GPU do pamięci hosta (procesora).
float sum = 0;
 
for (int i=0; i < N; i++) sum += a_h[i];
 //#sumujemy wartości z pamięci hosta i zapisujemy wynik do zmiennej sum.
sum *= 1.0 / (float)N;;
 //#dzielimy sumę przez liczbę elementów, aby uzyskać średnią.
free(a_h); cudaFree(a_d);
 //# zwalniamy pamięć hosta i GPU.
return sum;
 //# ta linijka zwraca wartość sumy, czyli wartość całki.
}

int main(void){
    FILE *fp = fopen("prostokatgpu1000000.txt", "w");
    cudaEvent_t start, stop;
 
    float x;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    //#tworzymy dwa zdarzenia - start i stop.
    cudaEventRecord(start, 0);
 //# zapisujemy moment uruchomienia programu w zdarzeniu start.
    x = licz(1000000);
    cudaEventRecord(stop, 0);
 //# zapisujemy moment zakończenia programu w zdarzeniu stop.
    cudaEventSynchronize(stop);
 //#synchronizujemy zdarzenie stop, aby być pewnym, że wszystkie operacje zostały wykon
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    float seconds = milliseconds / 1000;
    printf("czas wykonywania: %lf s\n", seconds);
    printf("wartość całki: %f" , x) ;
    fprintf(fp, "%f\n", seconds);
    fclose(fp);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    return 0 ; 
} 

# CPU - trapezy

In [ ]:
%%cu
#include<time.h>
#include<stdio.h>

float dokladnosc(int N)
{
    float sum = 0;
    for(int i = 0; i < N-1; ++i)
    {
        float x = (float)i/(float) N;
        float nastepny = (float)(i+1)/(float) N;
        float fx = (x*x+x+2);
        float fx_nastepny = ((nastepny*nastepny+nastepny+2) );
        sum += (fx + fx_nastepny)/2;
    }
    sum *= 1.0f / (float)N;
    return sum;
}
int main(){
    FILE *fp = fopen("trapezycpu1000000.txt", "w");
    clock_t start, stop;
    float calka ; 
    start = clock();
    calka = dokladnosc(1000000);
    stop = clock();
    printf("wartość całki: %f\n", calka);
    printf("czas wykonywania: %lf s", (double)(stop - start) / CLOCKS_PER_SEC);
    fprintf(fp, "%f\n", (double)(stop - start) / CLOCKS_PER_SEC);
    fclose(fp);
    return 0 ; 
}

# GPU - trapezy

In [ ]:
%%cu 
#include<stdio.h>
#include <cuda_runtime.h>
__global__ void integratorKernel(float *a, int N, float h)
{
    int idx = blockIdx.x * blockDim.x + threadIdx#.x;
    float x = (float)idx / (float)N;
    if(idx < N)
    a[idx] = (x*x#+x+2);
}
float licz(int N)
{
    size_t size = N * sizeof(float);
    float* a_h = (float# *)malloc(size);
    float* a_d ; cudaMalloc((void **) &a_d, size);
    int block_size# = 256;
    int n_blocks = N/block_size + (N % block_si#ze == 0 ? 0:1);
    float h = 1.0f / (float)N;
    integratorKernel <<< n_blocks, bl#ock_size >>> (a_d, N, h);
    cudaMemcpy(a_h, a_d, sizeof(float)*N, cudaMemcpyDeviceTo#Host);
    float sum = (a_h[0] + a_h[N-1]) / 2.0f;
    for (int i=1; i < N-1; i++) sum += a_h#[i];
    sum *= h;
    free(a_h); cudaFree(a_d);
    re#turn sum;
}
int main(void){
     FILE *fp = fopen("trapezygpu1000000.txt", "w");
     cudaE#vent_t start, stop;
     cudaEventCreate(&start); 
     #cudaEventCreate(&stop);
     cudaEventRecord(start, 0);
     float x = licz(1000000);
     cudaE#ventRecord(stop, 0);
     cudaEventSynchronize(stop);
     float milliseconds = 0;
     cudaEventElapsedTime(&millisec#onds, start, stop);
     float seconds = milliseconds / 1000; 
     printf("czas wykonywani#a: %lf s\n", seconds);
     printf("wartość całki: %f" , x) ;
     fprintf(fp, "%f\n", seconds);
     fclose(fp);
     cudaEve#ntDestroy(start);
     cudaEventDestroy(stop); 
     
     return 0 ; 

}// Deklaracja funkcji wywoływanej na GPU
__global__ // Deklaracja funkcji wywoływanej na GPU
__global__ void integratorKernel(float *a, int N, float h)
{
    // Indeks wątku
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // Obliczanie argumentu funkcji dla danego wątku
    float x = (float)idx / (float)N;

    // Wartość funkcji dla danego argumentu
    if(idx < N)
    a[idx] = (x*x+x+2);
}

// Funkcja obliczająca całkę
float licz(int N)
{
    // Rozmiar tablicy przechowującej wyniki
    size_t size = N * sizeof(float);

    // Alokacja pamięci na host (komputer)
    float* a_h = (float *)malloc(size);

    // Alokacja pamięci na device (GPU)
    float* a_d ; cudaMalloc((void **) &a_d, size);

    // Rozmiar bloku wątków
    int block_size = 256;

    // Liczba bloków wątków
    int n_blocks = N/block_size + (N % block_size == 0 ? 0:1);

    // Krok całkowania
    float h = 1.0f / (float)N;

    // Wywołanie funkcji na GPU
    integratorKernel <<< n_blocks, block_size >>> (a_d, N, h);

    // Kopiowanie wyniku z GPU do pamięci na host
    cudaMemcpy(a_h, a_d, sizeof(float)*N, cudaMemcpyDeviceToHost);

    // Inicjalizacja sumy wyników dla wątków
    float sum = (a_h[0] + a_h[N-1]) / 2.0f;

    // Sumowanie wyników dla wszystkich wątków (oprócz pierwszego i ostatniego)
    for (int i=1; i < N-1; i++) sum += a_h[i];

    // Mnożenie sumy przez krok całkowania
    sum *= h;

    // #Zwolnienie pamięci na host
    free(a_h); 

    // #Zwolnienie pamięci na GPU
    cudaFree(a_d);

    // #Zwracanie wyniku całkowania
    return sum;
}


# CPU - Simpson

In [ ]:
%%cu
#include<time.h>
#include<stdio.h>
float dokladnosc(int N)
{
    float sum = 0;
    for(int i = 0; i < N-1; i+=1)
    {
        float x = (float)i/(float) N;
        float srodkowy = (float)(i+1)/(float) N;
        float nastepny = (float)(i+2)/(float) N;
        float fx = (x*x+x+2);
        float fx_srodkowy = (srodkowy*srodkowy+srodkowy+2);
        float fx_nastepny = (nastepny*nastepny+nastepny+2);
        sum +=(fx + 4*fx_srodkowy + fx_nastepny)/6;
}
sum *= 1.0f/(float)N;
return sum;
}

int main(){
    FILE *fp = fopen("simpsoncpu1000000.txt", "w");
    clock_t start, stop;
    float calka ; 
    start = clock();
    calka = dokladnosc(1000000);
    stop = clock();
    printf("wartość całki: %f\n", calka);
    printf("czas wykonywania: %lf s", (double)(stop - start) / CLOCKS_PER_SEC);
    fprintf(fp, "%f\n", (double)(stop - start) / CLOCKS_PER_SEC);
    fclose(fp);
return 0 ; 
}


# GPU - Simpson

In [ ]:
%%cu 
#include<stdio.h>
#include <cuda_runtime.h>
//# Deklaracja funkcji GPU integratorKernel
//# a - wskaźnik na tablicę przechowującą wartości funkcji
//# N - liczba punktów, których wartości są obliczane
//# h - odległość pomiędzy dwoma sąsiednimi punktami__global__ void integratorKernel(float *a, int N, float h)
{

    //# Indeks wątku    int idx = blockIdx.x * blockDim.x + threadIdx.x;

     // #Obliczenie wartości x dla danego punktu    float x = (float)idx / (float)N;

    // Jeśli indeks jest mniejszy od liczby punktów, oblicz wartość funkcji    if(idx < N)
    a[idx] = (x*x+x+2);
}

// #Funkcja obliczająca wartość całki numeryczniefloat licz(int N)
{

    //# Przydzielenie pamięci na hosta    size_t size = N * sizeof(float);
    float* a_h = (float *)malloc(size);

    //# Przydzielenie pamięci na urządzenie GPU    float* a_d ; cudaMalloc((void **) &a_d, size);

    //# Określenie rozmiaru bloku    int block_size = 256;

    //# Określenie liczby bloków    int n_blocks = N/block_size + (N % block_size == 0 ? 0:1);

    //# Obliczenie wartości h    float h = 1.0f / (float)N;

    //# Wywołanie funkcji GPU integratorKernel    integratorKernel <<< n_blocks, block_size >>> (a_d, N, h); 

    //# Skopiowanie danych z GPU do pamięci hosta    cudaMemcpy(a_h, a_d, sizeof(float)*N, cudaMemcpyDeviceToHost);

     // #Inicjalizacja sumy    float sum = a_h[0] + a_h[N-1];

    fofor (int i=1; i < N-1; i++)
    {
        //# W każdej iteracji obliczamy wartość funkcji na punkcie x
        //# i dodajemy ją do sumy
        if (i % 2 == 0)
        //# Dla parzystych indeksów mnożymy przez 2
        sum += 2 * a_h[i];
        else
        //# Dla nieparzystych mnożymy przez 4
        sum += 4 * a_h[i];
    }
    //# Dzielimy sumę przez 3 i mnożymy przez h 
    //# (czyli przez krok całkowania)
    sum *= h / 3.0f;
    //# Zwalniamy pamięć na komputerze hosta
    free(a_h); 
    //# Zwalniamy pamięć na GPU
    cudaFree(a_d);
    //# Zwracamy wynik całkowania
    return sum;}
int main(void){
    FILE *fp = fopen("simpsongpu1000000.txt", "w");
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start, 0);
    float x = licz(1000000);
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);
    float seconds = milliseconds / 1000;
    printf("czas wykonywania: %f s\n", seconds);
    printf("wartość całki: %f" , x) ;
    fprintf(fp, "%f\n", seconds);
    fclose(fp);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    return 0 ; 

}
